# 0. Pre-Requisite Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt
import warnings
warnings.filterwarnings("ignore")
from functools import reduce
from pyod.models.abod import ABOD
from sklearn.preprocessing import MinMaxScaler

# 1. Dataset Processing

In [3]:
def pre_processing(df_original):
    '''
        INPUT: Original Dataset
        OUTPUT: Processed Dataset that is ready to be further used
        
        This is a function to calculate daily, total monthly rates and other featrues if they were not present
        ** The input dataset should have any of the following columns:
        [well_name - report_date - qo_bpd/monthly_produced_oil - qw_bpd/monthly_produced_water]
        ** Presence of [qg_mscfd/monthly_produced_gas - wor - wc_fraction - gor - reservoir_name - days] is optional;
                                                                                                                    '''
    df = df_original.copy()
    # In case the online days in month are not given, we assume they produce everyday
    if 'days' not in df.columns:
        df.loc[:, 'days'] = df.loc[:, 'report_date'].dt.days_in_month
        
    # Daily Rates Calculations
    if 'qo_bpd' not in df.columns:
        df.loc[:, 'qo_bpd']    = df.loc[:, 'monthly_produced_oil'] / df.loc[:, 'days']
    if 'qw_bpd' not in df.columns:
        df.loc[:, 'qw_bpd']  = df.loc[:, 'monthly_produced_water'] / df.loc[:, 'days']
    if 'qg_mscfd' not in df.columns:
        df.loc[:, 'qg_mscfd']    = df.loc[:, 'monthly_produced_gas'] / df.loc[:, 'days']
    if 'ql_bpd' not in df.columns:
        df.loc[:, 'ql_bpd']  = df.loc[:, 'qo_bpd'] + df.loc[:, 'qw_bpd'] 

      
    # to avoid division by zero
    df['qw_bpd'] = df['qw_bpd'].astype(np.float64)
    df['qo_bpd'] = df['qo_bpd'].astype(np.float64)
    df['qg_mscfd'] = df['qg_mscfd'].astype(np.float64)
    
    # Other Caclucations
    if 'wor' not in df.columns:
        df.loc[:, 'wor'] = df.loc[:, 'qw_bpd'] / df.loc[:, 'qo_bpd']
    if 'wc_fraction' not in df.columns:
        df.loc[:, 'wc_fraction']= df.loc[:, 'qw_bpd'] / df.loc[:, 'ql_bpd']
    if 'gor' not in df.columns:
        df.loc[:, 'gor']= df.loc[:, 'qg_mscfd'] * 1000 / df.loc[:, 'qo_bpd']
        
    # Dropping duplicated rows
    df.drop_duplicates(inplace = True)
    
    # Converting dates from strings into date time series
    df['report_date'] = pd.to_datetime(df['report_date'])
    # To avoid any date duplicated by mistake as entered by two different days on the same month
    df = df[df['report_date'].dt.day == 1]
    
    # Adding a feature for commingled reservoirs
    if 'reservoir_name' in df.columns:
        df['well_reservoir_name'] = df['well_name'] + '_' + df['reservoir_name']
    else:
        df['well_reservoir_name'] = df['well_name']
        
    # Dropping_duplicates for wells that have duplicated production date for the same reservoir
    df.drop_duplicates(subset = ['report_date', 'well_reservoir_name'], inplace = True)  

    # Dropping columns that are no more in use
    try:
        df.drop(['monthly_produced_oil', 'monthly_produced_water', 'monthly_produced_gas'],
                axis = 1, inplace = True)
    except KeyError:
        pass
    try:
        df.drop(['well_name'],
                axis = 1, inplace = True)
    except KeyError:
        pass    
    try:
        df.drop(['reservoir_name'],
                axis = 1, inplace = True)
    except KeyError:
        pass    
    
    df_preprocessed = df.copy()

    return (df_preprocessed)

In [5]:
def processing_well_data(df_preprocessed, wellname):
    ''' 
        INPUTS:
                df: pre-processed dataframe
                wellname: well to extract data of
        OUTPUT:
                dataframe for the well sorted and chan plot parameters calcualted
                                                                                                                    '''
    df = df_preprocessed.copy()
    # Sorting data by reporting dates
    well = df[df['well_reservoir_name'] == wellname].sort_values(by = 'report_date').reset_index(drop = True)
    # Total days of production
    well.loc[:, 'actual_days_of_production'] = (well.loc[:, 'report_date'] - well.loc[0,'report_date']).apply(lambda x: x.days)
    # Normalized days of production assuming all months of 30 days to match all wells on that column
    well['producing_days'] = ((well.loc[:, 'actual_days_of_production'].diff() // 28) * 30).cumsum()
    # Calculating derivative of WOR
    well["wor'"] = well.loc[:, 'wor'].diff() /  well.loc[:, 'actual_days_of_production'].diff()
    
    # Setting the first day of production as day number one
    well.loc[0, 'producing_days'] = 1
    # Setting name of online producction days
    well.rename(columns= {'days': 'online_days'}, inplace = True)
    
    # Cumulative calculations
    well['cumulative_oil_production'] = (well['qo_bpd']*well['online_days']).cumsum()
    
    # Setting days as index to match on
    well.set_index('producing_days', inplace = True)
    # Dropping unnecessary columns of deltas calculated
    well.drop(['actual_days_of_production'], axis = 1, inplace = True)
    
    # Setting top level column multiindex of wells names
    well.columns = pd.MultiIndex.from_product([[wellname], well.columns])

    well_processed = well.copy()
    return(well_processed)

In [6]:
def extract_full_dataset(df_preprocessed):
    '''
    INPUT:
    df: dataset that includes all data of wells
    OUTPUT:
    all_wells: dataframe that includes all wells ready to be chan plotted
                                                                                                                            '''
    df = df_preprocessed.copy()
    # unique well names considering commingled wells
    wellames = df['well_reservoir_name'].unique()
    # Setting mepty list to append dataframes into
    wells = []
    for wellname in wellames:
        # to show progress
        print(str(wellname) + ' ' + 'is now being processed')
        # Pre-processing using a previous function
        well_data = processing_well_data(df, wellname)
        # Appening the output table
        wells.append(well_data)
    # Concating all datasets on same vertical column as all wells have the same days intervals and empty rows are NaNs
    full_wells_data = pd.concat(wells, axis = 1)
    
    return(full_wells_data)

# 2. Outliers Removal

In [8]:
def outliers_algorithm(df_wrangled, mad_alpha):
    '''
        INPUT:
            df_wrangled: Scaled wrangled dataframe that only includes features to be used for outliers removal
        OUTPUT:
            preds: predictions either 0s or 1s
                                                                                                    '''
    df = df_wrangled.copy()
    ### Outliers Removal Algorithms
    model = ABOD(n_neighbors = 6, contamination = 0.15)

    # fitting models
    model_fit = model.fit(df)
    scores = model_fit.decision_scores_
    preds = model_fit.predict(df)
    
    return(preds)

In [9]:
def plotting_parameters(df_wrangled, original_days, preds):
    '''
        INPUT:
            df_scaled: original dataset scaled
            original_days: days not scaled
            preds: predictions either outliers or inliers
        OUTPUT:
            unique_clean_days: days that are classified as clean
            unique_outlier_days: days that are classified as outliers
                                                                                                                '''
    
    # Making a copy of the dataframe to avoid overriding new columns
    df_new = df_wrangled.copy()
    # df_new is the same as df_plot_full_cols but NaNs are filled with zeros in case features other than Qo to be used
    # in the model # Adding original days to avoid it being considered into the algorithm
    df_new['original_days']= original_days    
    
    # clean indices, clean dataframe scaled, and clean days
    clean_ind = np.where(pd.DataFrame(preds) == 0)[0].tolist()
    df_plot_clean_unique = df_new[df_new.index.isin(clean_ind)]
    unique_clean_days = df_plot_clean_unique['original_days']

    # outlier indices, outlier dataframe scaled, and outlier days
    outlier_ind = np.where(pd.DataFrame(preds) == 1)[0].tolist()
    df_plot_outliers_unique = df_new[df_new.index.isin(outlier_ind)]
    unique_outlier_days = df_plot_outliers_unique['original_days']
    
    return(unique_clean_days, unique_outlier_days)

In [10]:
def plot_inliers_vs_outliers(df, well_name, main_feature, df_plot_full_clean, df_plot_full_outlier):
    '''
        INPUT:
            df: original dataframe
            well_name: the proposed working well
            df_plot_full_cols: the full dataframe of the proposed well 
            unique_clean_days: clean indices
            unique_outlier_days: outlier indices
        OUTPUT:
            df_plot_full_clean: clean dataset of the well
            Three plots with full data, inlier data, outlier data
                                                                                                                    '''
    
    # Plotting axes
    x_col_plot = 'producing_days'
    y_col_plot = main_feature
    # Full table for original scatter plotting that includes the oiriginal data imported from excel sheet
    df_full = df.copy()
    df_full = df_full[well_name].reset_index()
    # Plot section
    fig = plt.figure(figsize = (25,5))
    ax = fig.add_subplot(1,3,1)
    plt.scatter(x = df_full[x_col_plot], y= df_full[y_col_plot])
    plt.title('Original Plot Imported')
    plt.xlabel(x_col_plot)
    plt.ylabel(y_col_plot) 
    
    ax1 = fig.add_subplot(1,3,2, sharex = ax, sharey = ax)
    plt.scatter(x = df_plot_full_clean[x_col_plot], y= df_plot_full_clean[y_col_plot], color = 'g', label = 'inliers')
    plt.scatter(x = df_plot_full_outlier[x_col_plot], y= df_plot_full_outlier[y_col_plot], color = 'red', label = 'outliers')
    plt.title('Original Plot Classified')
    plt.xlabel(x_col_plot)
    plt.ylabel(y_col_plot) 
    
    ax2 = fig.add_subplot(1,3,3, sharex = ax, sharey = ax)
    plt.scatter(x = df_plot_full_clean[x_col_plot], y= df_plot_full_clean[y_col_plot], color = 'g', label = 'inliers')
    plt.title('Clean Plot Output')
    plt.xlabel(x_col_plot)
    plt.ylabel(y_col_plot) 

In [ ]:
def celan_outliers(full_wells_data, threshold = 12, main_feature = 'qo_bpd', outlier_features = 'qo_bpd',
                                  show_outliers = False, mad_alpha = 2.5):
    '''
        INPUT:
            full_wells_data: Dataframe ready for chan plots
            main feature: feature to drop other columns if it is NA
            outlier_features: features to be used in outliers removal algorithm
        OUTPUT:
            df_clean: DataFrame ready after outliers removal  
                                                                                                                '''
    df = full_wells_data.copy()
    clean_wells = []
    for i, well_name in enumerate(df.columns.get_level_values(0).unique().tolist()):
        # New dataframe for each well & replace inf
        df_well = df[well_name].replace([np.inf, -np.inf], np.nan)
        # keeping points with available oil production only
        df_well = df_well[df_well[main_feature].notna()].reset_index()
        print(i, well_name)    
        if ((df_well['wor'].count() > threshold) and (df_well["wor'"].count() > threshold)):
            # Original production days
            original_days = df_well['producing_days']
            # the full dataset to be used in plotting that includes features not used in modelling
            df_plot_full_cols = df_well.copy()

            # Creating features to be used in outliers removal
            if type(outlier_features) != list:
                outlier_features_list = ['producing_days']
                outlier_features_list.append(outlier_features)
            elif type(outlier_features) == list:
                outlier_features_list = outlier_features
                outlier_features_list.append('producing_days')
            # Final dataset to be used in outliers removal
            df_well = df_well[outlier_features_list]

            # Scaling dataset with MinMax technique
            df_scaled = pd.DataFrame(MinMaxScaler().fit_transform(df_well), columns = df_well.columns)  
            # Final output
            df_wrangled = df_scaled.copy()


            # Calculating predictions
            preds = outliers_algorithm(df_wrangled, mad_alpha)
            # Original clean and outlier days
            unique_clean_days, unique_outlier_days = plotting_parameters(df_wrangled, original_days, preds)

            # clean and outlier dataframes according to algorithm 
            df_plot_full_clean = df_plot_full_cols[df_plot_full_cols['producing_days'].isin(unique_clean_days)]
            df_plot_full_outlier = df_plot_full_cols[df_plot_full_cols['producing_days'].isin(unique_outlier_days)] 

            if show_outliers != False:
                # Plotting original vs clean data
                plot_inliers_vs_outliers(df, well_name, main_feature, df_plot_full_clean, df_plot_full_outlier)

            df_plot_full_clean.set_index('producing_days', inplace = True)
            df_plot_full_clean.columns = pd.MultiIndex.from_product([[well_name], df_plot_full_clean.columns])
            clean_wells.append(df_plot_full_clean)

        else:
            continue

    clean_wells_df = reduce(lambda df_left,df_right: pd.merge(df_left, df_right, 
                                              left_index=True, right_index=True, 
                                              how='outer'), clean_wells)
        
    return(clean_wells_df)

# 3. Chan Plots

In [ ]:
def chan_plot(df):
    '''
        INPUT:
            original or clean dataframes
        OUTPUT:
            chan plot using original dataset
            chan plot with prediction using clean dataset
                                                                                                                        '''
    for i,well_name in enumerate(df.columns.get_level_values(0).tolist()):
        df_well = df[well_name]
        plt.ioff()
        fig = plt.figure(figsize = (8,8))
        # Chan plot for the selected well
        plt.scatter(x = df_well.index.tolist(), y= df_well['wor'], label = 'wor', c = 'blue', alpha = 0.3);
        plt.scatter(x = df_well.index.tolist(), y= df_well["wor'"], label = "wor'", marker = 'h', c = 'red', alpha = 0.3);
        # Transforming into log-log scale
        plt.xscale('log'), plt.yscale('log')
        # Setting limits and legend
        plt.xlim([1e0, 2e4])
        plt.xticks([1e0, 1e1, 1e2, 1e3, 1e4], ['1E0', '1E1', '1E2', '1E3', '1E4'])
        plt.ylim([1e-6, 1e4])
        plt.yticks([1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4], ['1E-6', '1E-5', '1E-4', '1E-3', '1E-2', '1E-1', '1E0', '1E1', '1E2', '1E3', '1E4'])
        #plt.axis('off')
        plt.grid()
        plt.title(well_name + ' ' + 'Chan Plot')
        plt.legend()
        plt.savefig('{}.png'.format(well_name));
        print(str(well_name) + ' ' + 'is diagnosed with Chan plots')

# 4. Main Function

In [11]:
def main_function(filename, outlier_features, main_feature = 'Qo_bpd', plot_outliers_option = False):
    '''
        INPUT:
            Dataset should include the following columns:
            ReportDate - WellName - ProducingDays
            MonthlyProducedOil "STB"/ MonthlyOilRate "STBD"- MonthlyProducedWater "BBL"/ MonthlyWaterRate "BPD"-
            MonthlyProducedGas "MSCF"/ MonthlyGasRate "MSCFD"
            -- Optional [MonthlyProducedLiquid "BBL" / MonthlyLiquidRate "BPD"- WC_fraction - WOR - GOR - WellStatus
            - ProductionMethod - Reservoir_name "Mandatory for Commingled Production"]
        
        OUTPUT:
            Full dataset for Chan plot and other necessary data that is ready to be plotted after outliers removal
                                                                                                                        '''
    # Reading excel sheet
    df = pd.read_excel(filename)
    # Pre-processing dataset
    df_preprocessed = pre_processing(df)
    # Extracting the final dataset
    full_wells_data = extract_full_dataset(df_preprocessed)
    # Remove outliers
    clean_df = celan_outliers(full_wells_data)
    # Diagnostic Plotting
    chan_plot(clean_df)
    
    return(clean_df)

# 5. Model Training

YOLO was used to train the model using data of 10,000 wellsfrom fields in Alaska, North Dakota, Colorado, and Gulf of Mexico.\
We used public avaialble datasets, and the sources of data were mentioned in the paper.\
The developed model was capable of achieving 0.85 mAP on the testing dataset.

# 6. Contact Information

We would appreciate your feedback on osama.abdelaziem@gmail.com \
To get the trained model or for further information, please contact me on the same mentioned email. 

# THANK YOU